In [1]:
import logging
import io
import sys
from googleads import adwords
import time
from time import gmtime, strftime
from urllib.request import urlopen
from urllib.parse   import quote
from urllib.error import HTTPError
import urllib.request
import random
from random import randint
import warnings
from progressbar import *
import progressbar
from multiprocessing.dummy import Pool as ThreadPool


#to-install libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pandas
from openpyxl import load_workbook

#to ignore warnings in terminal
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("data/report.csv")

In [3]:
regxp = re.compile('\["([\s\S]*)"\]')
def urlextract(urlrow):
    return re.findall(regxp, urlrow)[0]

In [4]:
data_s = data[:10]

In [5]:
data_s[:10]["Final URL"] = data_s[:10]["Final URL"].apply(urlextract)

In [6]:
USER_AGENT = 'User-agent'
ACCEPT = 'Accept'
ACCEPT_ENCODING = 'Accept-Encoding'
ACCEPT_LANGUAGE = 'Accept-language'
CONNECTION = 'Connection'

CONNECTION_DEFAULT = "keep-alive"
ACCEPT_ENCODING_DEFAULT = "utf-8"
ACCEPT_LANGUAGE_DEFAULT = 'en-us,en;q=0.5'

USER_AGENT_MAC_FIREFOX = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/45.0.2"

HEADER_FIREFOX_MAC = {USER_AGENT: USER_AGENT_MAC_FIREFOX,
    ACCEPT: "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        ACCEPT_ENCODING: ACCEPT_ENCODING_DEFAULT,
            ACCEPT_LANGUAGE: ACCEPT_LANGUAGE_DEFAULT,
                CONNECTION: CONNECTION_DEFAULT}

HEADER_CHROME_WINDOWS = {USER_AGENT: "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
    ACCEPT: "application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5",
        ACCEPT_ENCODING: ACCEPT_ENCODING_DEFAULT,
            ACCEPT_LANGUAGE: ACCEPT_LANGUAGE_DEFAULT,
                CONNECTION: CONNECTION_DEFAULT}

USER_HEADERS = [HEADER_FIREFOX_MAC, HEADER_CHROME_WINDOWS]


# In[152]:

# Random User-Agent string
def get_random_header():
    return USER_HEADERS[randint(0,(len(USER_HEADERS)-1))]

# Opener with random header settings
def get_random_opener():
    opener = urllib.request.build_opener()
    randomHeader = getRandomHeader()
    opener.addheaders = [(USER_AGENT,randomHeader[USER_AGENT]),
                         (ACCEPT,randomHeader[ACCEPT])
                         ,(ACCEPT_ENCODING,randomHeader[ACCEPT_ENCODING])
                         , (ACCEPT_LANGUAGE, randomHeader[ACCEPT_LANGUAGE])
                         ,(CONNECTION,randomHeader[CONNECTION])
                         ]
    return opener

In [7]:
def getBsObj(siteUrl, delayMin=0.01, delayMax=0.01):
    opener = getRandomOpener()

    time.sleep(random.uniform(delayMin, delayMax))
    # do not convert / and : chars as it ruins the 'http://' part of the url string

    try:
        r = requests.get(siteUrl)
        if r.status_code == 404:
            results_count = '404 error'
            return results_count
        html = opener.open(siteUrl)
        bsObj = BeautifulSoup(html.read(), "html.parser")
        bsObj_string = str(bsObj)
        if 'product result count' in bsObj_string:
            regxp = re.compile("<!--.* ([0-9]*)-->")
            results_count = re.findall(regxp,bsObj_string )
            results_count = [item for item in results_count if len(item) !=0 ]
            results_count = results_count[0]
        elif 'We’re Sorry.' in bsObj_string:
            results_count = "404 error"
        else:
            results_count = "Working"
    except:
        results_count = "recheck"
    return results_count

In [8]:
from bs4 import BeautifulSoup, Comment

def results_count(url):
    try:
        resp = requests.get(url)
        if r.status_code == 404:
            newlist = '404 error'
            return newlist
        soup = BeautifulSoup(resp.text, 'lxml')
        soup_string = str(soup)
        if 'We’re Sorry.' in bsObj_string:
            return "404 error"
        
        comments = soup.findAll(text=lambda text:isinstance(text, Comment))
        comments = [item.strip() for item in comments]
        pat = re.compile("product result count: ([0-9])*")
        newlist = [i for i in comments if pat.match(i)]
    except Exception as e:
        print(e)
        newlist = "re check"
    return newlist

In [11]:
data_s["Final URL"][0]

'https://www.overstock.com/Home-Garden/Bar-Stools/Rustic,/style,/2023/subcat.html?'

In [179]:
data_s["Final URL results"] = data_s["Final URL"].apply(results_count)

KeyboardInterrupt: 

In [193]:
str(data_s["Final URL results"][0])

"['product result count: 339']"

In [200]:
pat = re.compile("product result count: ([0-9]*)")
re.findall(pat, "product result count: 12")

['12']

In [198]:
def clean_results(row):
    row

### Different header generation

In [30]:
url = "https://www.overstock.com/Home-Garden/Bar-Stools/Rustic,/style,/2023/subcat.html?"
url2 = "https://www.overstock.com/Bedding-Bath/product.html?recalg=881,880,821,833&recidx=4&recset=5dec6555-3d23-4d00-b933-c2b9ad74218f&refccid=B2CT73EZBJJYLTSENV6OJ7ZLEI"

In [37]:
resp = requests.get(url)

In [38]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [39]:
soup

<!DOCTYPE html>

<html>
<!-- Copyright 2011 Overstock.com -->
<!-- all rights reserved -->
<!-- j25.prod.ostk.com ssl:true -->
<head>
<title>Rustic Bar Stools - Shop The Best Deals For Jun 2017</title>
<meta content="Rustic Bar Stools : Stylish bar stools provide a sense of authenticity and comfort to your home bar or kitchen counter experience. Free Shipping on orders over $45!" name="description"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<link href="https://www.overstock.com/Home-Garden/Bar-Stools/Rustic,/style,/2023/subcat.html" rel="canonical"/>
<meta content="index,follow" name="robots"/>
<link href="https://www.overstock.com/Home-Garden/Bar-Stools/Rustic,/style,/2023/subcat.html?index=61" rel="next"/>
<!-- SITE_03_HEAD (default) -->
<!-- SITE_HEAD_META (default) -->
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="Overstock™" name="author"/>
<meta content="NOODP" name="robots"/>
<!-- SITE_DNS_PREFETCH -->
<meta content="on" http-eq

In [ ]:
soup.find_all("div",{'class':''})

In [36]:
soup.find_all("div", {'class':'errTitle'})

[<div class="errTitle">We’re Sorry.</div>]

### Steps

In [91]:
USER_AGENT = 'User-agent'
ACCEPT = 'Accept'
ACCEPT_ENCODING = 'Accept-Encoding'
ACCEPT_LANGUAGE = 'Accept-language'
CONNECTION = 'Connection'

CONNECTION_DEFAULT = "keep-alive"
ACCEPT_ENCODING_DEFAULT = "utf-8"
ACCEPT_LANGUAGE_DEFAULT = 'en-us,en;q=0.5'

USER_AGENT_MAC_FIREFOX = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/45.0.2"

HEADER_FIREFOX_MAC = {USER_AGENT: USER_AGENT_MAC_FIREFOX,
  ACCEPT: "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        ACCEPT_ENCODING: ACCEPT_ENCODING_DEFAULT,
           ACCEPT_LANGUAGE: ACCEPT_LANGUAGE_DEFAULT,
                CONNECTION: CONNECTION_DEFAULT}

HEADER_CHROME_WINDOWS = {USER_AGENT: "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
    ACCEPT: "application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5",
        ACCEPT_ENCODING: ACCEPT_ENCODING_DEFAULT,
            ACCEPT_LANGUAGE: ACCEPT_LANGUAGE_DEFAULT,
                CONNECTION: CONNECTION_DEFAULT}

USER_HEADERS = [HEADER_FIREFOX_MAC, HEADER_CHROME_WINDOWS]

In [94]:
def get_random_header():
    return USER_HEADERS[randint(0,(len(USER_HEADERS)-1))]

In [98]:
t = get_random_header()

In [99]:
def results_count(url):
    try:
        resp = requests.get(url,t)
        if resp.status_code == 404:
            count = 0
            return newlist
        soup = BeautifulSoup(resp.text, 'lxml')
        soup_string = str(soup)
        if 'We’re Sorry.' in soup_string:
            return 0
        
        comments = soup.findAll(text=lambda text:isinstance(text, Comment))
        comments = [item.strip() for item in comments]
        pat = re.compile("product result count: ([0-9]+)") 
        newlist = [i for i in comments if pat.match(i)]
        count = re.findall("([0-9]+)",str(newlist))
        count = count[0]
    except Exception as e:
        print(e)
        count = "re check"
    return count

In [100]:
results_count(url)

'342'

In [101]:
import pandas as pd

In [103]:
t = pd.read_csv("data/report.csv")

In [115]:
t.apply(DataPull.url_extract, axis =1)

TypeError: ("url_extract() missing 1 required positional argument: 'row'", 'occurred at index 0')

In [109]:
from urlchecker import DataPull, UrlCheck

In [108]:
get_data = DataPull(report_type="camp")

In [ ]:
check = UrlCheck(list(campaign["FinalURL"]))

In [106]:
start = time.ctime()
print(start)


start = time.ctime()
print(start)

Wed Jun 28 15:11:16 2017
Wed Jun 28 15:11:16 2017


In [28]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
 
fromaddr = 'Broken URL Alerts <url_checker@overstock.com>'
toaddr = 'Manoj Dobbali <mdobbali@overstock.com>'
 
msg = MIMEMultipart()
 
msg['From'] = fromaddr
msg['To'] =  toaddr
msg['Subject'] = "SUBJECT OF THE EMAIL"
 
body = "TEXT YOU WANT TO SEND"
 
msg.attach(MIMEText(body, 'plain'))
 
filename = "report.csv"
attachment = open("data/report.csv", "rb")
 
part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
 
msg.attach(part)
 
server = smtplib.SMTP('outlook.overstock.com')
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()

(221, b'2.0.0 Service closing transmission channel')

In [ ]:
subject = 'Broken URL Alert'
msg = self.format

msg = MIMEText(msg, 'plain')
msg['Subject'] = subject

# Send the message via our own SMTP server.
s = smtplib.SMTP('outlook.overstock.com')
s.send_message(msg)
s.quit()

In [1]:
from urlchecker import EmailAlert

In [2]:
import pandas as pd

results_df  =  pd.read_csv("data/sample_report_camp.csv", index_col= 0)
print(results_df.head())

                                            FinalURL  \
0  https://www.overstock.com/Home-Garden/Chair-Pa...   
1  https://www.overstock.com/Home-Garden/Sofas-Lo...   
2  https://www.overstock.com/search?keywords=wood...   
3  https://www.overstock.com/Home-Garden/trundle-...   
4  https://www.overstock.com/Home-Garden/Headboar...   

                                     CustomParameter  \
0  {"cidmapping":"247234","dssagcrid":"lid=437000...   
1  {"cidmapping":"247234","dssagcrid":"lid=437000...   
2  {"cidmapping":"247234","dssagcrid":"lid=437000...   
3  {"cidmapping":"247234","dssagcrid":"lid=437000...   
4  {"cidmapping":"247234","dssagcrid":"lid=437000...   

                               AdGroup  CampaignID  \
0  Cushions|Rocking Chair Cushion|(CP)    50899044   
1                Sofa | Affordable (Q)    50899044   
2                Stepstools | Wood (A)    50899044   
3                        Bed | Trundle    50899044   
4          Headboards | Twin Size (CP)    50899044   

 

In [3]:
email = EmailAlert(["mdobbali@overtock.com"])
email.summarize(results_df)

FinalURL           object
CustomParameter    object
AdGroup            object
CampaignID          int64
Campaign           object
Results             int64
dtype: object


(10, 9, 0, 0, 0)

In [27]:
pd.DataFrame.from_records([(10, 9, 0 ,5, 0, 0)],columns = ["idenitfier" ,"total_urls", "unique_urls", "total_ads_with_broken_urls","unique_broken_urls", "recheck_urls"])

,idenitfier,total_urls,unique_urls,total_ads_with_broken_urls,unique_broken_urls,recheck_urls
0,10,9,0,5,0,0


In [36]:
"datasplit.csv".split("/")[-1]

'datasplit.csv'

In [37]:
"site" in "website"

True

### Campaign Index

In [8]:
import pandas as pd
from googleads import adwords
import logging

In [11]:
def pull_data(report=None,filename='data/summary.csv',skip_report_header=True, skip_column_header=False, skip_report_summary=True):
    """
    Creating API connection and pulls reports into CSV in data folder in current directory

    Parameters
    ----------
    filename : str
        filename is a path where data from API needs to be stored.
    report : json
        report is json formatted report definition for Google AdWords API. When it is none, it takes report definition which pulls data for all campaigns.
    skip_report_header: Boolean
        When skip_report_header is True, data pulled will not have report name. By default it is true.
    skip_column_header: Boolean
        When skip_column_header is True, data pulled will not have column names. By default it is False
    skip_report_summary: Boolean
        When skip_report_summary is True, data pulled will not have summary stats of report in last line. By default it is True

    """
    logging.basicConfig(level=logging.INFO)
    logging.getLogger('suds.transport').setLevel(logging.DEBUG)
    yaml = "/Users/mdobbali/Google Drive/2017/Work/googleads.yaml"
    client = adwords.AdWordsClient.LoadFromStorage(yaml)
    chunk_size = 16 * 1024
    report_downloader = client.GetReportDownloader(version='v201702')

    print("pulling data from API started...")


    # Create report definition.
    if report == None:
        report = {
              'reportName':'AdGroupReport',
              'dateRangeType': 'YESTERDAY',
              'reportType':'KEYWORDS_PERFORMANCE_REPORT',
              'downloadFormat': 'CSV',
              'selector': {
                        'fields':['CampaignId','CampaignName'],
#                             'predicates': [#{'field': 'CampaignId', 'operator': 'IN', 'values': [50899044]},
#                                            {'field': 'AdGroupStatus',
#                                             'operator': 'IN',
#                                             'values': ['ENABLED', 'PAUSED']}
#                                 ]
                    }
                 }

    stream_data = report_downloader.DownloadReportAsStream(report, skip_report_header=skip_report_header, skip_column_header=skip_column_header,skip_report_summary=skip_report_summary)
    f = open(filename, 'wb')
    try:

        while True:
            chunk = stream_data.read(chunk_size)
            if not chunk:
                break
            f.write(chunk)
    finally:
        stream_data.close()
        f.close()

In [12]:
pull_data()

INFO:oauth2client.client:Refreshing access_token
DEBUG:suds.transport.http:opening (https://adwords.google.com/api/adwords/reportdownload/v201702/reportDefinition.xsd)
DEBUG:suds.transport.http:opening (https://adwords.google.com/api/adwords/reportdownload/v201702/reportDefinition.xsd)


pulling data from API started...


In [14]:
df = pd.read_csv("data/summary.csv")

In [18]:
df = df.drop_duplicates(keep="first").reset_index()

In [19]:
df.to_csv("data/summary.csv")

### Mapping

In [31]:
import pandas as pd 
import glob
import re

In [3]:
list_of_files = glob.glob("data/campaign_reports/*.csv")

In [12]:
master_df = pd.DataFrame()
for file in list_of_files:
    df = pd.read_csv(file, index_col=0)
    master_df = master_df.append(df, ignore_index = True)

In [18]:
len(list(set(master_df["FinalURL"])))

70994

In [41]:
site_links = pd.read_csv("data/campaign_reports/site_links.csv")

In [50]:
reg_expression = re.compile('\["([\s\S]*)"\]')

In [54]:
def url_extract(row):
    """
    An apply function for DataFrame which extracts urls from the report

    Parameters
    ----------
    row : DataFrame

    """
    try:
        row["FinalURL"] = re.findall(reg_expression, row["FinalURL"])[0]
    except:
        print(row)
    return row

In [60]:
site_links = site_links.apply(url_extract, axis=1)